### Nutzung der Helper-Klasse

In [111]:
from db_communication import db_communication
import pandas as pd
import numpy as np

In [112]:
config = {
    'user': 'root',
    'password': 'FDS-apm1',
    'host': 'min-ifm-xdm.ad.fh-bielefeld.de',
    'port': '3306',
    'database': 'NeueDB' 
}
my_db = db_communication(config)

Successfully connected.


In [113]:
tables = my_db.read_tables()
tables = [tables[i][0] for i in range(len(tables))]
tables

['Auftrag',
 'Einzelteile',
 'Konfigurator',
 'Kunde',
 'Merkmalcluster',
 'Merkmale',
 'Stueckliste']

In [114]:
for table in tables:
    print(table)
    print(my_db.show_columns(table))

Auftrag
[('AuftrNr', 'int(11)', 'NO', 'PRI', None, 'auto_increment'), ('Bestelldatum', 'date', 'YES', '', None, ''), ('Lieferdatum', 'date', 'YES', '', None, ''), ('KundenNr', 'int(11)', 'NO', 'MUL', None, ''), ('Gesamtpreis', 'decimal(7,2)', 'YES', '', None, '')]
Einzelteile
[('EinzelteilNr', 'int(11)', 'NO', 'PRI', None, 'auto_increment'), ('Bezeichnung', 'char(35)', 'NO', '', None, ''), ('Preis', 'decimal(7,2)', 'YES', '', None, '')]
Konfigurator
[('KonfigNr', 'int(11)', 'NO', 'PRI', None, 'auto_increment'), ('AuftragNr', 'int(11)', 'NO', 'MUL', None, ''), ('ArtikelNr', 'int(11)', 'NO', 'MUL', None, '')]
Kunde
[('KundenNr', 'int(11)', 'NO', 'PRI', None, 'auto_increment'), ('Name', 'char(30)', 'NO', '', None, ''), ('Vorname', 'char(30)', 'NO', '', None, ''), ('Mail', 'char(60)', 'YES', '', None, '')]
Merkmalcluster
[('MerkmalNr', 'int(11)', 'NO', 'PRI', None, 'auto_increment'), ('Merkmalsbezeichnung', 'char(35)', 'YES', '', None, '')]
Merkmale
[('ArtNr', 'int(11)', 'NO', 'PRI', None,

### Funktionen nach Kategorie des zufälligen Wertes: 

In [115]:
import random
def randomElementInList(values):
    """
    Auswahl eines zufälligen Elements aus dem Wertebereich values
    Bsp.: ['Basic','Holland','BMX','Klassisch','Einrad'] ->  return = 'BMX'
    """
    return random.choice(values)

In [116]:
randomElementInList(['Basic','Holland','BMX','Klassisch','Einrad'])

'Klassisch'

In [117]:
def randomNumber(start,stop=None,step=1):
    """
    Generierung einer zufälligen Zahl nach random.randrange():
    Choose a random item from range(start, stop[, step])
    """
    return random.randrange(start=start,stop=stop,step=step)

In [118]:
randomNumber(1,20,1)

12

In [119]:
import time 
import datetime
def randomDate(start, end=datetime.datetime.now().strftime("%d.%m.%Y"), format="%d.%m.%Y", prop=random.random()):
    """Get a time at a proportion of a range of two formatted times.

    start and end should be strings specifying times formated in the
    given format (strftime-style), giving an interval [start, end].
    prop specifies how a proportion of the interval to be taken after
    start.  The returned time will be in the specified format.
    """

    stime = time.mktime(time.strptime(start, format))
    etime = time.mktime(time.strptime(end, format))

    ptime = stime + prop * (etime - stime)

    return time.strftime(format, time.localtime(ptime))

In [120]:
randomDate("12.10.2020","12.10.2022","%d.%m.%Y",random.random())

'04.08.2021'

In [121]:
def randomName():
    preNames = ["Tana","Francoise","Emmie","Stuart","Cedrick","Tiffani","Kym","Woodrow","Jolene","Autumn","Moriah","Claud","Antoinette",
                "Gisele","Chrissy","Ellena","Hilton","Susanne","Lindy","Nakesha","Rebecca","Lavenia","Donya","Claudia","Dreama","Alla",
                "Freeman","Weston","Jacki","Helene"]
    lastName = ["Allinder","Longley","Dobyns","Coldiron","Justin","Munroe","Boll","Clingerman","Pries","Santangelo","Miller","Bohman","Meachum",
                "Kamer","Waits","Polin","Suitt","Saulter","Pelt","Dapolito","Guild","Beeks","Hollingshead","Schissler","Cardamone","Sypher",
                "Fannin","Belizaire","Burleigh","Eslinger"]
    return randomElementInList(preNames), randomElementInList(lastName)

In [122]:
def randomMail(preName,lastName):
    providers = ['gmail.com','web.de','telekom.de','apple.com','fh-bielefeld.de']
    mail = "{0}.{1}@{2}".format(preName.lower(),lastName.lower(),randomElementInList(providers).lower())
    return mail

## Fill data

### Kundendaten

In [ ]:
def fillCustomerTable(n,columnNamesDB=['Name','Vorname','Mail'],includeID=True):
    """
    Hinzufügen von zufällig generierten Einträgen in der DB-Tabelle 'Kunde'
    
    n - Anzahl der Einträge die hinzugefügt werden sollen
    columnNamesDB - Spaltennamen der Kunden-Tabelle in der DB (mit Ausnahme der KundenNr; Reihenfolge beachten!) 
    includeID - Ob KundenNr seperat mit angelegt werden soll
    
    Es werden die Inhalte nach den defaults in den Parametern erzeugt!
    Das Ändern der Reihenfolge der Spalten und die Hinzunahme weiterer Spalten muss im Quellcode angepasst werden!
    """
    # Kundentabelle füllen
    table_name = 'Kunde'
    customerId = 'KundenNr'
    if includeID:
        customer_keys = np.concatenate(([customerId],columnNamesDB)) 
    else:
        customer_keys = columnNamesDB
    for i in range(n):
        customer_name = randomName()
        customer_values = i+1, customer_name[0], customer_name[1], randomMail(*customer_name)
        if includeID:
            customer_dict = dict(zip(customer_keys,customer_values))
        else:
            customer_dict = dict(zip(customer_keys,customer_values[1:]))
        my_db.insert_data(table_name,customer_dict)

In [ ]:
fillCustomerTable(1,includeID=False)

### Konfiguraton

In [ ]:
# TODO

### Aufträge

In [ ]:
def fillOrderTable(n,columnNamesDB=['Datum','KundenNr'],includeID=True,
                   customerTableName='Kunde',customerId='KundenNr',startDate='2020-01-01'):
    """
    Hinzufügen von zufällig generierten Einträgen in der DB-Tabelle 'Aufträge'
    
    n - Anzahl der Einträge die hinzugefügt werden sollen
    columnNamesDB - Spaltennamen der Auftrag-Tabelle in der DB (mit Ausnahme der AuftrNr; Reihenfolge beachten!) 
    includeID - Ob AuftrNr seperat mit angelegt werden soll
    customerTableName - Name der DB-Tabelle 'Kunde'
    customerId - Name der Identifikationsnummer in customerTable
    startDate - Startdatum zur zufälligen Erzeugung
    
    Es werden die Inhalte nach den defaults in den Parametern erzeugt!
    Das Ändern der Reihenfolge der Spalten und die Hinzunahme weiterer Spalten muss im Quellcode angepasst werden!
    """
    table_name = 'Auftrag'
    orderId = 'AuftrNr'
    
    # Lesen der Wertebereiche aus relevanten Tabellen
    df_customers = my_db.get_table(customerTableName)
    
    # Auftragstabelle füllen
    if includeID:
        order_keys = np.concatenate(([customerId],columnNamesDB)) 
    else:
        order_keys = columnNamesDB
    for i in range(n):
        rand_orderDate = randomDate(start=startDate,
                                    end=datetime.datetime.now().strftime("%Y-%m-%d"),
                                    format="%Y-%m-%d",
                                    prop=random.random())
        rand_customerId = randomNumber(start=df_customers[customerId].astype(int).min(),
                                        stop=df_customers[customerId].astype(int).max())
        order_values = i+1, rand_orderDate, rand_customerId
        if includeID:
            order_dict = dict(zip(order_keys,order_values))
        else:
            order_dict = dict(zip(order_keys,order_values[1:]))
        my_db.insert_data(table_name,order_dict)

In [ ]:
fillOrderTable(1)

### Konfigurator TEST

In [10]:
# define Merkmalscluster
merkmalcluster  = [
    {'ClusterNr':'1','Merkmalsbezeichnung':'Rahmen'},
    {'ClusterNr':'2','Merkmalsbezeichnung':'Gabel'},
    {'ClusterNr':'3','Merkmalsbezeichnung':'Lackierung'},
    {'ClusterNr':'4','Merkmalsbezeichnung':'Lenker'},
    {'ClusterNr':'5','Merkmalsbezeichnung':'Felgen'},
    {'ClusterNr':'6','Merkmalsbezeichnung':'Radbereifung'},
    {'ClusterNr':'7','Merkmalsbezeichnung':'Gangschaltung'},
    {'ClusterNr':'8','Merkmalsbezeichnung':'Sattel'},
    {'ClusterNr':'9','Merkmalsbezeichnung':'Griffe'},
    {'ClusterNr':'10','Merkmalsbezeichnung':'Bremsen'},
]

In [11]:
merkmale = [ 
    #Rahmen
    { 'MerkmalNr':'100001','Merkmalcluster':'1','Bezeichnung': 'FH100','Preis':'499.00','Montagezeit': '2','Ausstattung':'City'},
    { 'MerkmalNr':'100002','Merkmalcluster':'1','Bezeichnung': 'S100','Preis':'649.99','Montagezeit': '2','Ausstattung':'Sport'},
    { 'MerkmalNr':'100003','Merkmalcluster':'1','Bezeichnung': 'Trofeo 5','Preis':'599.00','Montagezeit': '2','Ausstattung':'Rennen'},
    
    { 'MerkmalNr':'199999','Merkmalcluster':'1','Bezeichnung': 'Kein Rahmen','Preis':'0.00','Montagezeit': '0','Ausstattung':'NULL'},
    #Gabel
    { 'MerkmalNr':'200001','Merkmalcluster':'2','Bezeichnung': 'Aluminium Citystoßdämpfer','Preis':'9.99','Montagezeit': '1','Ausstattung':'City'},
    { 'MerkmalNr':'200002','Merkmalcluster':'2','Bezeichnung': 'Carbon Öldruckstoßdämpfer','Preis':'49.00','Montagezeit': '2','Ausstattung':'Sport'},
    { 'MerkmalNr':'200003','Merkmalcluster':'2','Bezeichnung': 'Carbon starr','Preis':'29.00','Montagezeit': '1','Ausstattung':'Rennen'},
    
    { 'MerkmalNr':'299999','Merkmalcluster':'2','Bezeichnung': 'Keine Gabel','Preis':'0.00','Montagezeit': '0','Ausstattung':'NULL'},
    #Lackierung/ Farbe
    { 'MerkmalNr':'300001','Merkmalcluster':'3','Bezeichnung': 'Schwarz uni','Preis':'10.00','Montagezeit': '1','Ausstattung':'NULL'},
    { 'MerkmalNr':'300002','Merkmalcluster':'3','Bezeichnung': 'Weiß uni','Preis':'15.00','Montagezeit': '1','Ausstattung':'NULL'},
    { 'MerkmalNr':'300003','Merkmalcluster':'3','Bezeichnung': 'Saphirschwarz metallic','Preis':'29.00','Montagezeit': '1','Ausstattung':'NULL'},
    { 'MerkmalNr':'300004','Merkmalcluster':'3','Bezeichnung': 'Mineralweiß metallic','Preis':'29.00','Montagezeit': '1','Ausstattung':'NULL'},
    { 'MerkmalNr':'300005','Merkmalcluster':'3','Bezeichnung': 'Phytonicblau','Preis':'50.00','Montagezeit': '1','Ausstattung':'NULL'},
    { 'MerkmalNr':'300006','Merkmalcluster':'3','Bezeichnung': 'Aventurinrot II metallic','Preis':'65.00','Montagezeit': '2','Ausstattung':'NULL'},
    { 'MerkmalNr':'300007','Merkmalcluster':'3','Bezeichnung': 'Frozen Bluestone metallic','Preis':'70.00','Montagezeit': '2','Ausstattung':'NULL'},
    
    { 'MerkmalNr':'399999','Merkmalcluster':'3','Bezeichnung': 'Keine Lackierung','Preis':'0.00','Montagezeit': '0','Ausstattung':'NULL'},
    #Lenker
    { 'MerkmalNr':'400001','Merkmalcluster':'4','Bezeichnung': 'Retro Lenker','Preis':'89.99','Montagezeit': '1','Ausstattung':'Sport'},
    { 'MerkmalNr':'400002','Merkmalcluster':'4','Bezeichnung': 'Classic Lenker','Preis':'59.49','Montagezeit': '1','Ausstattung':'NULL'},
    { 'MerkmalNr':'400003','Merkmalcluster':'4','Bezeichnung': 'Rennradlenker','Preis':'99.99','Montagezeit': '1','Ausstattung':'Rennen'},
    
    { 'MerkmalNr':'499999','Merkmalcluster':'4','Bezeichnung': 'Kein Lenker','Preis':'0.00','Montagezeit': '0','Ausstattung':'NULL'},
    #Felgen
    { 'MerkmalNr':'500001','Merkmalcluster':'5','Bezeichnung': 'Aluminium Felgen','Preis':'10.00','Montagezeit': '3','Ausstattung':'NULL'},
    { 'MerkmalNr':'500002','Merkmalcluster':'5','Bezeichnung': 'Redondo Rennradfelgen','Preis':'99.99','Montagezeit': '4','Ausstattung':'Rennen'},
    { 'MerkmalNr':'500003','Merkmalcluster':'5','Bezeichnung': 'Spengle Carbonfelgen','Preis':'79.99','Montagezeit': '2','Ausstattung':'NULL'},
    { 'MerkmalNr':'500004','Merkmalcluster':'5','Bezeichnung': 'Taylor Sportfelgen','Preis':'39.99','Montagezeit': '5','Ausstattung':'Sport'},
    { 'MerkmalNr':'500005','Merkmalcluster':'5','Bezeichnung': 'Taylor Cityfelgen','Preis':'69.99','Montagezeit': '5','Ausstattung':'City'},
    
    { 'MerkmalNr':'599999','Merkmalcluster':'5','Bezeichnung': 'Keine Felge','Preis':'0.00','Montagezeit': '0','Ausstattung':'NULL'},
    #Radbereifung
    { 'MerkmalNr':'600001','Merkmalcluster':'6','Bezeichnung': 'Bereifung Schwalbe City','Preis':'15.00','Montagezeit': '2','Ausstattung':'City'},
    { 'MerkmalNr':'600002','Merkmalcluster':'6','Bezeichnung': 'Bereifung Schwalbe Sport','Preis':'20.00','Montagezeit': '1','Ausstattung':'Sport'},
    { 'MerkmalNr':'600003','Merkmalcluster':'6','Bezeichnung': 'Bereifung Schwalbe Marathon','Preis':'35.00','Montagezeit': '4','Ausstattung':'Rennen'},
    
    { 'MerkmalNr':'699999','Merkmalcluster':'6','Bezeichnung': 'Keine Reifen','Preis':'0.00','Montagezeit': '0','Ausstattung':'NULL'},
    #Gangschaltung
    { 'MerkmalNr':'700001','Merkmalcluster':'7','Bezeichnung': '3 Gang Nexus RT','Preis':'100.00','Montagezeit': '1','Ausstattung':'NULL'},
    { 'MerkmalNr':'700002','Merkmalcluster':'7','Bezeichnung': '7 Gang Nexus RT','Preis':'110.00','Montagezeit': '1','Ausstattung':'NULL'},
    { 'MerkmalNr':'700003','Merkmalcluster':'7','Bezeichnung': '9 Gang Nexus RT','Preis':'150.00','Montagezeit': '1','Ausstattung':'NULL'},
    
    { 'MerkmalNr':'799999','Merkmalcluster':'7','Bezeichnung': 'Keine Schaltung','Preis':'0.00','Montagezeit': '0','Ausstattung':'NULL'},
    #Sattel
    { 'MerkmalNr':'800001','Merkmalcluster':'8','Bezeichnung': 'Asvert Cityradsattel','Preis':'10.00','Montagezeit': '1','Ausstattung':'NULL'},
    { 'MerkmalNr':'800002','Merkmalcluster':'8','Bezeichnung': 'Asvert Comfortsattel','Preis':'39.00','Montagezeit': '1','Ausstattung':'NULL'},
    { 'MerkmalNr':'800003','Merkmalcluster':'8','Bezeichnung': 'Avert Rennradsattel','Preis':'59.00','Montagezeit': '1','Ausstattung':'Rennen'},
    
    { 'MerkmalNr':'899999','Merkmalcluster':'8','Bezeichnung': 'Kein Sattel','Preis':'0.00','Montagezeit': '0','Ausstattung':'NULL'},
    #Griffe
    { 'MerkmalNr':'900001','Merkmalcluster':'9','Bezeichnung': 'Girff schwarz Weichgummi','Preis':'11.99','Montagezeit': '1','Ausstattung':'NULL'},
    { 'MerkmalNr':'900002','Merkmalcluster':'9','Bezeichnung': 'Griff schwarz Hartgummi','Preis':'11.99','Montagezeit': '1','Ausstattung':'Rennen'},
    { 'MerkmalNr':'900003','Merkmalcluster':'9','Bezeichnung': 'Griff schwarz Kautschuk','Preis':'10.00','Montagezeit': '1','Ausstattung':'NULL'},
    { 'MerkmalNr':'900003','Merkmalcluster':'9','Bezeichnung': 'Griff braun Leder','Preis':'19.99','Montagezeit': '1','Ausstattung':'City'},
    
    { 'MerkmalNr':'999999','Merkmalcluster':'9','Bezeichnung': 'Keine Griffe','Preis':'0.00','Montagezeit': '0','Ausstattung':'NULL'},
    #Bremsen
    { 'MerkmalNr':'1000001','Merkmalcluster':'10','Bezeichnung': 'Shimano V-Brake','Preis':'10.00','Montagezeit': '1','Ausstattung':'NULL'},
    { 'MerkmalNr':'1000002','Merkmalcluster':'10','Bezeichnung': 'Clarks Scheibenbremse','Preis':'69.99','Montagezeit': '2','Ausstattung':'Sport'},
    { 'MerkmalNr':'1000003','Merkmalcluster':'10','Bezeichnung': 'Shimano GRB Scheibenbremse','Preis':'99.99','Montagezeit': '2','Ausstattung':'NULL'},
    { 'MerkmalNr':'1000004','Merkmalcluster':'10','Bezeichnung': 'Fagura Felgenbremse','Preis':'39.99','Montagezeit': '1','Ausstattung':'NULL'},
    
    { 'MerkmalNr':'1099999','Merkmalcluster':'10','Bezeichnung': 'Keine Bremse','Preis':'0.00','Montagezeit': '0','Ausstattung':'NULL'},
]

In [24]:
# TODO: replace this by read tabels from DB
df_merkmalcluster = pd.DataFrame(merkmalcluster)
df_merkmalcluster.set_index('ClusterNr',inplace=True)
df_merkmale = pd.DataFrame(merkmale)

In [56]:
#merge dfs
arrays = [df_merkmale['Merkmalcluster'].replace(df_merkmalcluster.to_dict()['Merkmalsbezeichnung']).values, 
          df_merkmale['MerkmalNr'].values]
tuples = list(zip(*arrays))
index = pd.MultiIndex.from_tuples(tuples, names=['Merkmalcluster', 'MerkmalNr'])
df = df_merkmale.set_index(index)
df

MerkmalNr Merkmalcluster  \
Merkmalcluster MerkmalNr                            
Rahmen         100001       100001              1   
               100002       100002              1   
               100003       100003              1   
               199999       199999              1   
Gabel          200001       200001              2   
               200002       200002              2   
               200003       200003              2   
               299999       299999              2   
Lackierung     300001       300001              3   
               300002       300002              3   
               300003       300003              3   
               300004       300004              3   
               300005       300005              3   
               300006       300006              3   
               300007       300007              3   
               399999       399999              3   
Lenker         400001       400001              4   
               400002       400002              4   
               400003       400003              4   
               499999       499999              4   
Felgen         500001       500001              5   
               500002       500002              5   
               500003       500003              5   
               500004       500004              5   
               500005       500005              5   
               599999       599999              5   
Radbereifung   600001       600001              6   
               600002       600002              6   
               600003       600003              6   
               699999       699999              6   
Gangschaltung  700001       700001              7   
               700002       700002              7   
               700003       700003              7   
               799999       799999              7   
Sattel         800001       800001              8   
               800002       800002              8   
               800003       800003              8   
               899999       899999              8   
Griffe         900001       900001              9   
               900002       900002              9   
               900003       900003              9   
               900003       900003              9   
               999999       999999              9   
Bremsen        1000001     1000001             10   
               1000002     1000002             10   
               1000003     1000003             10   
               1000004     1000004             10   
               1099999     1099999             10   

                                          Bezeichnung   Preis Montagezeit  \
Merkmalcluster MerkmalNr                                                    
Rahmen         100001                           FH100  499.00           2   
               100002                            S100  649.99           2   
               100003                        Trofeo 5  599.00           2   
               199999                     Kein Rahmen    0.00           0   
Gabel          200001       Aluminium Citystoßdämpfer    9.99           1   
               200002       Carbon Öldruckstoßdämpfer   49.00           2   
               200003                    Carbon starr   29.00           1   
               299999                     Keine Gabel    0.00           0   
Lackierung     300001                     Schwarz uni   10.00           1   
               300002                        Weiß uni   15.00           1   
               300003          Saphirschwarz metallic   29.00           1   
               300004            Mineralweiß metallic   29.00           1   
               300005                    Phytonicblau   50.00           1   
               300006        Aventurinrot II metallic   65.00           2   
               300007       Frozen Bluestone metallic   70.00           2   
               399999                Keine Lackierung    0.00    

In [88]:
for merkmal in df['Merkmalcluster'].unique():
    print('generate number for merkmal:',merkmal)

generate number for merkmal: 1
generate number for merkmal: 2
generate number for merkmal: 3
generate number for merkmal: 4
generate number for merkmal: 5
generate number for merkmal: 6
generate number for merkmal: 7
generate number for merkmal: 8
generate number for merkmal: 9
generate number for merkmal: 10


In [132]:
# get set of Merkmale
setMerkmale = df.query("Merkmalcluster == '8'")['MerkmalNr'].values
# ignoe 'no ...' part
unvalidClusters = [element for element in setMerkmale if '99999' in element]
setMerkmale = setMerkmale[setMerkmale != unvalidClusters]

In [136]:
# select random element
randomElementInList(setMerkmale)

'800003'

In [210]:
# build bike
bike = []
bikePrice = 0
for cluster in df['Merkmalcluster'].unique():
    # get set of Merkmale
    setMerkmale = df.query("Merkmalcluster == " + "'" + cluster + "'")['MerkmalNr'].values
    # ignoe 'no ...' part (marked with MerkmalNr : 'xxx99999')
    unvalidClusters = [element for element in setMerkmale if '99999' in element]
    setMerkmale = setMerkmale[setMerkmale != unvalidClusters]
    # select random bike part
    bikePart = randomElementInList(setMerkmale)
    # get price of this part
    pricePart = float(df.swaplevel().loc[str(bikePart)]['Preis'][0])
    # sum it up to get the total bike price
    bikePrice += pricePart
    bike.append(bikePart)
bike, bikePrice

(['100001',
  '200003',
  '300003',
  '400003',
  '500002',
  '600003',
  '700002',
  '800003',
  '900003',
  '1000002'],
 1040.97)

In [137]:
# TODO: generate insert